# 투 포인터

정렬된 배열에서 두 개의 인덱스(pointer)를 움직이면서 특정 조건을 만족하는 구간을 빠르게 탐색하는 알고리즘이다.

- 일반적 투 포인터 (양 끝에서 좁히는 방식) : 정렬된 배열에서 조건에 맞는 쌍 탐색
- 연속 구간형 (슬라이딩 윈도우) : 연속 구간(구간 합, 길이, 최대값, 최솟값) 탐색
- 조건 누적형 : 요소의 중복 허용 X
- 조합 최적화형 : 부분 집합의 조합 탐색
- 다중 포인터 : 여러 개의 포인터를 사용

In [1]:
# 투 포인터 (양 끝 수렴형) - 배열 내 두 수의 합이 target에 가장 가까운 쌍 탐색
def tow_pointer(arr, target):
  # 1. 정렬 및 포인터 초기화
  arr.sort()
  left, right = 0, len(arr)-1
  
  check = float('inf')
  result = ()
  
  # 2. 투 포인터 진행
  while left < right:
    total = arr[left] + arr[right]
    
    # 2.1. 조건 확인 : 조건에 맞는다면 결과 갱신
    if abs(total - target) < abs(check - target):
      check = total
      result = (arr[left], arr[right])
    
    # 2.2. 포인터 이동
    if total < target:
      left += 1
    else:
      right -= 1
    
  return result

print(tow_pointer([1, 3, 5, 7, 10], 11))

(1, 10)


In [2]:
# 연속 구간형 - 합이 target 이상인 가장 짧은 연속 부분 수열 위치 쌍 탐색
def sliding_window(arr, target):
  # 1. 왼쪽 포인터 초기화
  left = 0

  total = 0
  result = (-1, -1)
  min_len = float('inf')

  # 2. 오른쪽 포인터 이동
  for right in range(len(arr)):
    total += arr[right]

    # 3. 왼쪽 포인터 이동 : 왼쪽 포인터가 이동하며 조건에 맞는 결과 탐색
    while total >= target:
      now_len = right - left + 1
      if now_len < min_len:
        min_len = now_len
        result = (left, right)
      total -= arr[left]
      left += 1

  return result if result != (-1, -1) else None

print(sliding_window([1, 2, 3, 4, 5], 9))

(3, 4)


In [3]:
# 조건 누적형 - 중복이 없는 가장 긴 부분 수열의 길이
def condition_two_pointer(arr):
  # 1. 왼쪽 포인터 초기화
  left = 0

  seen = set()
  max_len = 0
  result = (0, 0)
  
  # 2. 오른쪽 포인터 이동
  for right in range(len(arr)):
    
    # 3. 왼쪽 포인터 이동 : 왼쪽 포인터가 이동하며 조건에 맞는 결과 탐색
    while arr[right] in seen:
      seen.remove(arr[left])
      left += 1

    seen.add(arr[right])

    # 정보 갱신
    now_len = right - left + 1 
    if now_len > max_len:
      max_len = now_len
      result = (left, right)

  return result

print(condition_two_pointer([1, 2, 3, 2, 4, 5]))

(2, 5)


In [4]:
# 조합 최적화형 - 부분집합 중 target과 일치하는 경우의 수 탐색
from itertools import combinations
from bisect import bisect_left, bisect_right

def get_all_sums(arr):
  # arr의 모든 부분집합의 합 반환
  result = []
  for i in range(len(arr)+1):
    for comb in combinations(arr, i):
      result.append(sum(comb))
  return result

def meet_in_middle(arr, target):
  # 1. 배열을 두 부분으로 분할 : 탐색 범위 감소
  passive_arr = arr[:len(arr)//2] # passive(left) : 조건 제공자(active를 도움)
  active_arr = arr[len(arr)//2:]  # active(right) : 정렬 후 이진 탐색으로 조건 판단

  # 2. 절반으로 나눈 배열으로 연산 : 연산 횟수 감소
  passive_sums = get_all_sums(passive_arr)
  active_sums = get_all_sums(active_arr)

  # 3. 하나의 배열만 정렬 : 정렬 범위를 줄임 (이진 탐색 사용)
  active_sums.sort()
  
  cnt = 0
  # 4. 반대 배열 값 순회 : 정렬된 배열과 연산하여 정렬 없이 값 도출
  for passive_sum in passive_sums:
    remain = target - passive_sum

    # 4.1. bisect_left : target(remain)이 처음으로 나타는 위치
    left = bisect_left(active_sums, remain)
    # 4.2. bisect_right : target(remain)보다 큰 값이 처음 나타나는 위치
    right = bisect_right(active_sums, remain)
    # 4.3. 목적에 맞는 값 산출
    cnt += right - left

  return cnt

arr = [1, 2, 3, 2, 4, 5]
target = 7
print(meet_in_middle(arr, target))

6


In [5]:
# 세 포인터 - 배열 내 세 수의 합이 target에 가장 가까운 쌍 탐색
def three_pointer(arr, target):
  # 1. 정렬
  arr.sort()

  result = ()
  check = float('inf')
  
  # 2. 왼쪽 포인터(1 pointer) 설정
  for left in range(len(arr)-2):
    # 2.1. 중앙 포인터(2 pointer), 오른쪽 포인터(3 pointer) 설정
    mid, right = left+1, len(arr)-1
    
    # 2.2. 투 포인터 실행
    while mid < right:
      total = arr[left] + arr[mid] + arr[right]

      # 2.2.1. 조건 확인
      if abs(total - target) < abs(check - target):
        check = total
        result = (arr[left], arr[mid], arr[right])

      # 2.2.2. 포인터 이동
      if total < target:
        mid += 1
      else:
        right -= 1
  
  return result

print(three_pointer([-1, 2, 1, -4], 1))

(-1, 1, 2)
